# 🍽️ Food-101 CNN Classifier (PyTorch + timm)
Colab-ready | Official split | AMP | Cosine LR | Grad-CAM


## 0. Check runtime (GPU?)

In [ ]:
import torch
print('CUDA available:', torch.cuda.is_available())
print('Device count:', torch.cuda.device_count())

In [ ]:
import kagglehub
kmader_food41_path = kagglehub.dataset_download('kmader/food41')

print('Data source import complete.')

## 1. Install libraries

In [ ]:
!pip -q install timm==0.9.16 seaborn scikit-learn torchinfo
!pip -q install git+https://github.com/jacobgil/pytorch-grad-cam.git


## 2. Imports & config

In [ ]:
import os, random, json
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.metrics import confusion_matrix, classification_report, top_k_accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from torchinfo import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# ---- Config ----
DATA_ROOT = Path('/kaggle/input/food41')
IMG_DIR   = DATA_ROOT / 'images'
META_DIR  = DATA_ROOT / 'meta'


BACKBONE = 'efficientnet_b3'
IMG_SIZE = 224
BATCH    = 64
EPOCHS   = 12
LR       = 3e-4
WEIGHT_DECAY = 1e-4
LABEL_SMOOTH = 0.1
NUM_WORKERS = 2
SEED = 42

torch.manual_seed(SEED); random.seed(SEED); np.random.seed(SEED)

In [ ]:
import os
print('Images exist:', IMG_DIR.exists())
print('Meta exist:', META_DIR.exists())
print('Sample classes:', sorted(os.listdir(IMG_DIR))[:5])


## 3. Dataset & loaders (official split)

In [ ]:
import json
from pathlib import Path

def read_txt_list(p):
    with open(p) as f:
        return [line.strip() for line in f]

# 1) Classes
if (META_DIR/'classes.txt').exists():
    classes = [c.strip() for c in open(META_DIR/'classes.txt')]
elif (META_DIR/'classes.json').exists():
    classes = json.load(open(META_DIR/'classes.json'))
else:
    # fallback: infer from folder names
    classes = sorted([d.name for d in IMG_DIR.iterdir() if d.is_dir()])
class_to_idx = {c:i for i,c in enumerate(classes)}

# 2) Train/Test lists
if (META_DIR/'train.txt').exists():
    train_list = read_txt_list(META_DIR/'train.txt')
    test_list  = read_txt_list(META_DIR/'test.txt')
elif (META_DIR/'train.json').exists():
    train_list = json.load(open(META_DIR/'train.json'))
    test_list  = json.load(open(META_DIR/'test.json'))
else:
    # fallback: random split
    from sklearn.model_selection import train_test_split
    all_files = [p.relative_to(IMG_DIR).with_suffix('').as_posix()
                 for p in IMG_DIR.rglob('*.jpg')]
    y = [p.split('/')[0] for p in all_files]
    train_list, test_list = train_test_split(all_files, test_size=0.2,
                                             random_state=42, stratify=y)

print(len(classes), 'classes')
print(len(train_list), 'train samples')
print(len(test_list),  'test samples')


In [ ]:
def read_split(txt_file):
    with open(txt_file) as f:
        return [line.strip() for line in f]

#  classes = [c.strip() for c in open(META_DIR/'classes.txt')]
#  class_to_idx = {c:i for i,c in enumerate(classes)}

#  train_list = read_split(META_DIR/'train.txt')
#  test_list  = read_split(META_DIR/'test.txt')

train_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    transforms.RandomErasing(p=0.1)
])

test_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

class Food101Split(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
    def __len__(self):
        return len(self.file_list)
    def __getitem__(self, idx):
        rel = self.file_list[idx]
        img_path = IMG_DIR/f'{rel}.jpg'
        y = class_to_idx[rel.split('/')[0]]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, y

train_ds = Food101Split(train_list, transform=train_tfms)
test_ds  = Food101Split(test_list,  transform=test_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

len(train_ds), len(test_ds)

## 5. Model & optimizer

In [ ]:
model = timm.create_model(BACKBONE, pretrained=True, num_classes=len(classes)).to(device)
summary(model, input_size=(BATCH,3,IMG_SIZE,IMG_SIZE), verbose=0)

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

In [ ]:
from pathlib import Path
Path('/content/best_food101.pth').exists()


## 6. Train & evaluate loops

In [ ]:
'model' in locals(), 'labels_tensor' in locals()


In [ ]:
def train_one_epoch(epoch):
    model.train()
    run_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            out = model(x)
            loss = criterion(out, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        run_loss += loss.item()
        pbar.set_postfix(loss=run_loss/len(pbar))
    scheduler.step()

@torch.no_grad()
def evaluate(loader):
    model.eval()
    correct, total = 0, 0
    all_logits, all_labels = [], []
    for x, y in loader:
        x = x.to(device)
        logits = model(x).cpu()
        preds = logits.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
        all_logits.append(logits)
        all_labels.append(y)
    acc = 100*correct/total
    all_logits = torch.cat(all_logits)
    all_labels = torch.cat(all_labels)
    top5 = top_k_accuracy_score(all_labels, all_logits, k=5, labels=range(len(classes)))
    print(f'Val Acc: {acc:.2f}%, Top-5: {top5*100:.2f}%')
    return acc, top5, all_logits.argmax(1), all_labels

In [ ]:
best = 0
for ep in range(1, EPOCHS+1):
    train_one_epoch(ep)
    acc, top5, preds, labels_tensor = evaluate(test_loader)
    if acc > best:
        best = acc
        torch.save(model.state_dict(), 'best_food101.pth')
        print('✅ Saved new best model')

## 7. Confusion matrix & report

In [ ]:
cm = confusion_matrix(labels_tensor, preds, normalize='true')
plt.figure(figsize=(12,10))
sns.heatmap(cm, cmap='magma', cbar=False)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

print(classification_report(labels_tensor, preds, target_names=classes[:len(set(labels_tensor))]))

## 8. Grad-CAM

In [ ]:
def show_gradcam(img_path, true_label=None, class_idx=None):
    model.eval()
    rgb_img = Image.open(img_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
    input_tensor = test_tfms(rgb_img).unsqueeze(0).to(device)

    # pick last conv layer automatically
    target_layers = [list(model.modules())[-2]]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

    targets = None
    if class_idx is not None:
        targets = [ClassifierOutputTarget(class_idx)]

    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]
    rgb_img_np = np.array(rgb_img)/255.0
    visualization = show_cam_on_image(rgb_img_np, grayscale_cam, use_rgb=True)

    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1); plt.imshow(rgb_img_np); plt.axis('off'); plt.title('Original')
    plt.subplot(1,2,2); plt.imshow(visualization); plt.axis('off'); plt.title('Grad-CAM')
    if true_label is not None:
        plt.suptitle(f'True: {true_label}')
    plt.show()

# Example
sample_rel = random.choice(test_list)
img_path = IMG_DIR/f'{sample_rel}.jpg'
true_lbl = sample_rel.split('/')[0]
show_gradcam(img_path, true_label=true_lbl)


## 9. Download artifacts

In [ ]:
from google.colab import files
files.download('best_food101.pth')